In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
# more info on callbakcs: https://keras.io/callbacks/ model saver is cool too.
from tensorflow.keras.callbacks import TensorBoard
from imblearn.over_sampling import RandomOverSampler
import pickle
import time

C:\Users\narah\anaconda3\lib\site-packages\scipy\__init__.py:177: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
data = pd.read_csv('../datasets/diabetes.csv')
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [4]:
data.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [6]:
data.shape

(768, 9)

In [ ]:
data.corr()

In [ ]:
import seaborn as sns
sns.heatmap(data)

In [ ]:
import matplotlib.pyplot as plt
sns.pairplot(data)
plt.figure(figsize=(15,15))
plt.show()

In [ ]:
X = data.iloc[:,:-1]
y = data['Outcome']

In [ ]:
y = np.reshape(y.values,(y.shape[0],1))

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)
new_data = np.hstack((X, np.reshape(y[:], (-1, 1))))
transformed_df = pd.DataFrame(data, columns = data.columns)

In [28]:
X.shape

(768, 8)

In [29]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=100)

In [30]:


dense_layers = [2]
layer_sizes = [32]


for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        model = Sequential()
        model.add(Activation('relu'))
        model.add(Flatten())

        for _ in range(dense_layer):
            model.add(Dense(layer_size))
            model.add(Activation('relu'))

        model.add(Dense(1))
        model.add(Activation('sigmoid'))
        NAME = "{}-nodes-{}-dense-{}".format( layer_size, dense_layer, int(time.time()))
        tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

        model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'],
                          )
        model.fit(X_train, y_train,batch_size=5,epochs=20,validation_split = 0.3,callbacks=[tensorboard])
        
        y_pred_prob = model.predict(X_val)
        y_pred = (y_pred_prob > 0.5).astype(int)


Epoch 1/20
97/97 [==============================] - 3s 9ms/step - loss: 0.6832 - accuracy: 0.6335 - val_loss: 0.6433 - val_accuracy: 0.7981
Epoch 2/20
97/97 [==============================] - 1s 7ms/step - loss: 0.6163 - accuracy: 0.7391 - val_loss: 0.5761 - val_accuracy: 0.7740
Epoch 3/20
97/97 [==============================] - 1s 7ms/step - loss: 0.5523 - accuracy: 0.7391 - val_loss: 0.5250 - val_accuracy: 0.7981
Epoch 4/20
97/97 [==============================] - 1s 6ms/step - loss: 0.5170 - accuracy: 0.7578 - val_loss: 0.5066 - val_accuracy: 0.7885
Epoch 5/20
97/97 [==============================] - 1s 6ms/step - loss: 0.5003 - accuracy: 0.7516 - val_loss: 0.4990 - val_accuracy: 0.7885
Epoch 6/20
97/97 [==============================] - 1s 6ms/step - loss: 0.4923 - accuracy: 0.7681 - val_loss: 0.4932 - val_accuracy: 0.7644
Epoch 7/20
97/97 [==============================] - 1s 6ms/step - loss: 0.4892 - accuracy: 0.7619 - val_loss: 0.4891 - val_accuracy: 0.7885
Epoch 8/20
97/97 [==

In [31]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_val, y_pred)

In [32]:
cm

array([[41,  6],
       [14, 16]], dtype=int64)

In [33]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 activation (Activation)     (None, 8)                 0         
                                                                 
 flatten (Flatten)           (None, 8)                 0         
                                                                 
 dense (Dense)               (None, 32)                288       
                                                                 
 activation_1 (Activation)   (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                1056      
                                                                 
 activation_2 (Activation)   (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 3

In [34]:
pickle.dump(model, open("./models/diabetic.pkl",'wb'))